In [1]:
# importing libraries
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd

In [2]:
# loading the dataset into pandas Dataframe
data=pd.read_csv(r"C:\Users\Rohit Mourya\Downloads\comments_data.csv")

In [3]:
# calling first five rows of datasets
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
# checking for null values in datasets
data.isnull().sum()

id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

In [5]:
# understanding structure of your dataset
data.shape

(159571, 8)

In [6]:
# cleaning and preprocessing text data i.e comment_text from DataFrame
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range(len(data['comment_text'])):
    review=re.sub('[^a-zA-Z0-9]',' ',data['comment_text'][i])
    review=review.lower()
#     review=review.split()
    
#     review=[ps.stem(words) for words in review]
#     review=' '.join(review)
    corpus.append(review)

In [7]:
corpus

['explanation why the edits made under my username hardcore metallica fan were reverted  they weren t vandalisms  just closure on some gas after i voted at new york dolls fac  and please don t remove the template from the talk page since i m retired now 89 205 38 27',
 'd aww  he matches this background colour i m seemingly stuck with  thanks    talk  21 51  january 11  2016  utc ',
 'hey man  i m really not trying to edit war  it s just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page  he seems to care more about the formatting than the actual info ',
 '  more i can t make any real suggestions on improvement   i wondered if the section statistics should be later on  or a subsection of   types of accidents     i think the references may need tidying so that they are all in the exact same format ie date format etc  i can do that later on  if no one else does first   if you have any preferences for formatting style on refer

In [8]:
# Encoding the preprocessed text using one_hot encoder
import tensorflow
from tensorflow.keras.preprocessing.text import one_hot
voc_size=20000
oh_rpr=[one_hot(words,voc_size) for words in corpus]

In [9]:
oh_rpr

[[11692,
  3255,
  16356,
  15542,
  17972,
  10082,
  7893,
  602,
  10523,
  5273,
  19999,
  10529,
  6874,
  19910,
  18723,
  13875,
  5488,
  1148,
  2887,
  18627,
  14068,
  4447,
  7709,
  19603,
  15063,
  9038,
  2402,
  3854,
  15293,
  11661,
  7926,
  4806,
  15224,
  13875,
  1886,
  16356,
  5521,
  6460,
  16356,
  5803,
  5885,
  4138,
  19603,
  6217,
  16444,
  10024,
  7903,
  17742,
  1110,
  15304],
 [10766,
  1613,
  1276,
  1390,
  15484,
  10029,
  14910,
  19603,
  6217,
  5595,
  1363,
  9769,
  6331,
  5803,
  9692,
  18235,
  8978,
  16311,
  9360,
  330],
 [11929,
  13556,
  19603,
  6217,
  4291,
  10961,
  7245,
  17301,
  5182,
  6513,
  5468,
  14842,
  1148,
  6146,
  15484,
  211,
  6053,
  11170,
  16380,
  16580,
  1542,
  7926,
  7452,
  17301,
  18195,
  14562,
  15542,
  19515,
  3247,
  7893,
  5803,
  5885,
  1276,
  16333,
  17301,
  18027,
  16418,
  11493,
  16356,
  12478,
  128,
  16356,
  18343,
  7906],
 [16418,
  19603,
  19577,
  138

In [10]:
# Padding the encoded sequences to ensure uniform length of every comment
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_docs=pad_sequences(oh_rpr,padding='pre',maxlen=50)

In [11]:
padded_docs

array([[11692,  3255, 16356, ..., 17742,  1110, 15304],
       [    0,     0,     0, ..., 16311,  9360,   330],
       [    0,     0,     0, ..., 16356, 18343,  7906],
       ...,
       [    0,     0,     0, ...,  5873, 14687,  3977],
       [    0,     0,     0, ..., 16629,  9038,  5468],
       [    0,     0,     0, ...,  4695,  9061,  2753]])

In [12]:
# separating the independent variable and dependent variable
x=padded_docs
y=data.iloc[:,2:].values

In [13]:
y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

In [14]:
# Splitting the data into training and testing sets.
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [15]:
# importing libraries for building deep learning model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,Dense,Bidirectional,LSTM

In [16]:
# Create a Sequential model using TensorFlow's Keras API.
model=Sequential()
# Adding Embedding layers in model
model.add(Embedding(20000,50))
# Adding Bidirectional LSTM layer
model.add(Bidirectional(LSTM(32,activation='tanh')))
# Adding Dense layer in model
model.add(Dense(128,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(6,activation='sigmoid'))

In [17]:
# Compiling the model with suitable optimizer and loss function
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [18]:
# Training the model on the training data for a specified number of epochs.
model.fit(x_train,y_train,validation_split=0.2,epochs=3)

Epoch 1/3
3192/3192 ━━━━━━━━━━━━━━━━━━━━ 78s 23ms/step - accuracy: 0.9442 - loss: 0.0989 - val_accuracy: 0.9949 - val_loss: 0.0596
Epoch 2/3
3192/3192 ━━━━━━━━━━━━━━━━━━━━ 71s 22ms/step - accuracy: 0.9842 - loss: 0.0507 - val_accuracy: 0.9949 - val_loss: 0.0581
Epoch 3/3
3192/3192 ━━━━━━━━━━━━━━━━━━━━ 71s 22ms/step - accuracy: 0.9942 - loss: 0.0421 - val_accuracy: 0.9949 - val_loss: 0.0638


In [19]:
# Summary of model
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 50, 50)              │       1,000,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 64)                  │          21,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 6)                   │             774 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,288,788 (12.55 MB)

 Trainable params: 1,096,262 (4.18 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,192,526 (8.36 MB)

In [20]:
# Evaluating the model's performance on the test data.
model.evaluate(x_test,y_test)

998/998 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9935 - loss: 0.0615


[0.06306039541959763, 0.99357670545578]

In [21]:
# Defining a function for predicting toxicity levls based on user input

def sentences(sent):     
    sent=sent.lower()
    text=[sent]

    ohr_rpr=[one_hot(words,voc_size) for words in text]
    pad_docs=pad_sequences(ohr_rpr,padding='pre',maxlen=50)
    z=model.predict(pad_docs)
    
    print('toxic:',round(z[0][0]*100),'%')
    print('severe_toxic:',round(z[0][1]*100),'%')
    print('obsence:',round(z[0][2]*100),'%')
    print('threat:',round(z[0][3]*100),'%')
    print('insult:',round(z[0][4]*100),'%')
    print('identity_hate::',round(z[0][5]*100),'%')
    
sent=input("Enter your comment:")
sentences(sent)

Enter your comment:They went to college.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 373ms/step
toxic: 5 %
severe_toxic: 0 %
obsence: 0 %
threat: 0 %
insult: 1 %
identity_hate:: 0 %


In [22]:
# Creating a Gradio interface with input as text and output as text, displaying toxicity predictions in terms of percentage.
import gradio as gr

def predict(text):
    text = text.lower()
    text = [text]

    ohr_rpr = [one_hot(words, voc_size) for words in text]
    pad_docs = pad_sequences(ohr_rpr, padding='pre', maxlen=50)
    z = model.predict(pad_docs)

    toxic = round(z[0][0] * 100,2)
    severe_toxic = round(z[0][1] * 100,2)
    obsence = round(z[0][2] * 100,2)
    threat = round(z[0][3] * 100,2)
    insult = round(z[0][4] * 100,2)
    identity_hate = round(z[0][5] * 100,2)

    report =f"Toxic: {toxic}%\nSevere Toxic: {severe_toxic}%\nObscene: {obsence}%\nThreat: {threat}%\nInsult: {insult}%\nIdentity Hate: {identity_hate}%"
    return report

# Creating interface for our model
interface = gr.Interface(
  fn=predict,
  inputs="text",
  outputs="text",
  title="Toxicity Analyzer",
  description="Enter a comment below to predict its toxicity levels.",
)
# Launching the Gradio interface for user interaction.
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7869
Running on public URL: https://104b312fe2e6191dca.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [23]:
# Creating a Gradio interface with input as text and output as text, displaying toxicity predictions.
import gradio as gr

def predict(text):
    text = text.lower()
    text = [text]

    ohr_rpr = [one_hot(words, voc_size) for words in text]
    pad_docs = pad_sequences(ohr_rpr, padding='pre', maxlen=50)
    z = model.predict(pad_docs)

    toxic = round(z[0][0] * 100, 2)
    severe_toxic = round(z[0][1] * 100, 2)
    obsence = round(z[0][2] * 100, 2)
    threat = round(z[0][3] * 100, 2)
    insult = round(z[0][4] * 100, 2)
    identity_hate = round(z[0][5] * 100, 2)
    
    if toxic>=30:
        toxic=True
    else:
        toxic=False
    if severe_toxic>=30:
        severe_toxic=True
    else:
        severe_toxic=False
    if obsence>=30:
        obsence=True
    else:
        obsence=False
    if threat>=30:
        threat=True
    else:
        threat=False
    if insult>=30:
        insult=True
    else:
        insult=False
    if identity_hate>=30:
        identity_hate=True
    else:
        identity_hate=False

    report =f"Toxic: {toxic}\nSevere Toxic: {severe_toxic}\nObscene: {obsence}\nThreat: {threat}\nInsult: {insult}\nIdentity Hate: {identity_hate}"
    return report

interface = gr.Interface(
  fn=predict,
  inputs="text",
  outputs="text",
  title="Toxicity Analyzer",
  description="Enter a comment below to predict its toxicity levels.",
)
#  Launching the Gradio interface for user interaction.
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7870
Running on public URL: https://8f68a48f841d811fda.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
